In [3]:
import numpy as np
from minst import loadData

trainData, trainLabel = loadData(fileName='D:/Jupyter/mnist_train.csv',data_bin=1,label_bin=1)
testData, testLabel   = loadData(fileName='D:/Jupyter/mnist_test.csv',data_bin=1,label_bin=1)

start read file
start read file


In [1]:
def calc_e_Gx(trainDataArr, trainLabelArr, n, div, rule, D):
    '''
    计算分类错误率
    :param trainDataArr:训练数据集数字
    :param trainLabelArr: 训练标签集数组
    :param n: 要操作的特征
    :param div:划分点
    :param rule:正反例标签
    :param D:权值分布D
    :return:预测结果， 分类误差率
    '''

    e = 0                    #初始化分类误差率为0
    x = trainDataArr[:, n]   #将训练数据矩阵中特征为n的那一列单独剥出来做成数组。因为其他元素我们并不需要，
    y = trainLabelArr        #同样将标签也转换成数组格式
    predict = []

    if rule == 'LisOne':   
        L = 1; H = -1     #依据小于和大于的标签依据实际情况会不同，在这里直接进行设置
    else:                  
        L = -1; H = 1
        
    for i in range(trainDataArr.shape[0]):    #遍历所有样本的特征m
        if x[i] < div:                        #如果小于划分点，则预测为L #如果设置小于div为1，那么L就是1， #如果设置小于div为-1，L就是-1
            predict.append(L)    
            if y[i] != L: 
                e += D[i]                     #如果预测错误，分类错误率要加上该分错的样本的权值（8.1式）
        elif x[i] >= div:      
            predict.append(H)                 #与上面思想一样
            if y[i] != H: 
                e += D[i]
    #返回预测结果和分类错误率e
    #预测结果其实是为了后面做准备的，在算法8.1第四步式8.4中exp内部有个Gx，要用在那个地方
    #以此来更新新的D
    return np.array(predict), e

In [4]:
def createSigleBoostingTree(trainDataArr, trainLabelArr, D):
    '''
    创建单层提升树
    :param trainDataArr:训练数据集数组
    :param trainLabelArr: 训练标签集数组
    :param D: 算法8.1中的D
    :return: 创建的单层提升树
    '''
    m, n = np.shape(trainDataArr)    #获得样本数目及特征数量
    sigleBoostTree = {}              #单层树的字典，用于存放当前层提升树的参数   
    sigleBoostTree['e'] = 1          #初始化分类误差率，分类误差率在算法8.1步骤（2）（b）有提到  #误差率最高也只能100%，因此初始化为1

    for i in range(n):                   #对每一个特征进行遍历，寻找用于划分的最合适的特征 
        for div in [-0.5, 0.5, 1.5]:     #因为特征已经经过二值化，只能为0和1，因此分切分时分为-0.5， 0.5， 1.5三挡进行切割
            
            #在单个特征内对正反例进行划分时，有两种情况：
            #可能是小于某值的为1，大于某值得为-1，也可能小于某值得是-1，反之为1
            #因此在寻找最佳提升树的同时对于两种情况也需要遍历运行
            #LisOne：Low is one：小于某值得是1
            #HisOne：High is one：大于某值得是1
            for rule in ['LisOne', 'HisOne']:
                #按照第i个特征，以值div进行切割，进行当前设置得到的预测和分类错误率
                Gx, e = calc_e_Gx(trainDataArr, trainLabelArr, i, div, rule, D) 
                if e < sigleBoostTree['e']:      #如果分类错误率e小于当前最小的e，那么将它作为最小的分类错误率保存
                    sigleBoostTree['e'] = e
                    sigleBoostTree['div'] = div   #同时也需要存储最优划分点、划分规则、预测结果、特征索引以便进行D更新和后续预测使用
                    sigleBoostTree['rule'] = rule
                    sigleBoostTree['Gx'] = Gx
                    sigleBoostTree['feature'] = i
    return sigleBoostTree

In [6]:
def createBosstingTree(trainDataList, trainLabelList, treeNum = 50):
    '''
    创建提升树
    创建算法依据“8.1.2 AdaBoost算法” 算法8.1
    :param trainDataList:训练数据集
    :param trainLabelList: 训练测试集
    :param treeNum: 树的层数
    :return: 提升树
    '''
    trainDataArr = np.array(trainDataList)      #将数据和标签转化为数组形式
    trainLabelArr = np.array(trainLabelList)
    finallpredict = [0] * len(trainLabelArr)    #每增加一层数后，当前最终预测结果列表
    m, n = np.shape(trainDataArr)               #获得训练集数量以及特征个数
    D = [1 / m] * m                             #依据算法8.1步骤（1）初始化D为1/N
    tree = []                                   #初始化提升树列表，每个位置为一层

    for i in range(treeNum):                                                  #循环创建提升树
        curTree = createSigleBoostingTree(trainDataArr, trainLabelArr, D)     #得到当前层的提升树
        alpha = 1/2 * np.log((1 - curTree['e']) / curTree['e'])               #根据式8.2计算当前层的alpha
        Gx = curTree['Gx']                                                    #获得当前层的预测结果，用于下一步更新D
        #依据式8.4更新D
        #np.multiply(trainLabelArr, Gx)：exp中的y*Gm(x)，结果是一个行向量，内部为yi*Gm(xi)
        #np.exp(-1 * alpha * np.multiply(trainLabelArr, Gx))：上面求出来的行向量内部全体成员再乘以-αm，然后取对数，和书上式子一样，
        #只不过书上式子内是一个数，这里是一个向量
        #D是一个行向量，取代了式中的wmi，然后D求和为Zm
        #书中的式子最后得出来一个数w，所有数w组合形成新的D
        #这里是直接得到一个向量，向量内元素是所有的w
        D = np.multiply(D, np.exp(-1 * alpha * np.multiply(trainLabelArr, Gx))) / sum(D)

        curTree['alpha'] = alpha        #在当前层参数中增加alpha参数，预测的时候需要用到

        tree.append(curTree)            #将当前层添加到提升树索引中。

        #-----以下代码用来辅助，可以去掉---------------

        finallpredict += alpha * Gx        #根据8.6式将结果加上当前层乘以α，得到目前的最终输出预测
    
        error = sum([1 for i in range(len(trainDataList)) if np.sign(finallpredict[i]) != trainLabelArr[i]])#计算当前预测输出与实际标签间误差
  
        finallError = error / len(trainDataList)      #计算当前最终误差率
  
        if finallError == 0:    
            return tree      #如果误差为0，提前退出即可，因为没有必要再计算算了

        print('iter:%d:%d, sigle error:%.4f, finall error:%.4f'%(i, treeNum, curTree['e'], finallError ))        #打印一些信息

    return tree    #返回整个提升树

In [8]:
tree = createBosstingTree(trainData[:1000], trainLabel[:1000], 40)

iter:0:40, sigle error:0.9030, finall error:1.0000
iter:1:40, sigle error:0.9030, finall error:1.0000
iter:2:40, sigle error:0.7532, finall error:1.0000
iter:3:40, sigle error:0.5000, finall error:1.0000
iter:4:40, sigle error:0.3641, finall error:1.0000
iter:5:40, sigle error:0.3641, finall error:1.0000
iter:6:40, sigle error:0.4307, finall error:1.0000
iter:7:40, sigle error:0.5000, finall error:1.0000
iter:8:40, sigle error:0.5348, finall error:1.0000
iter:9:40, sigle error:0.5348, finall error:1.0000
iter:10:40, sigle error:0.5174, finall error:1.0000
iter:11:40, sigle error:0.5000, finall error:1.0000
iter:12:40, sigle error:0.4913, finall error:1.0000
iter:13:40, sigle error:0.4913, finall error:1.0000
iter:14:40, sigle error:0.4956, finall error:1.0000
iter:15:40, sigle error:0.5000, finall error:1.0000
iter:16:40, sigle error:0.5022, finall error:1.0000
iter:17:40, sigle error:0.5022, finall error:1.0000
iter:18:40, sigle error:0.5011, finall error:1.0000
iter:19:40, sigle erro

In [9]:
tree


[{'e': 0.9030000000000007,
  'div': -0.5,
  'rule': 'HisOne',
  'Gx': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1,

In [7]:
def predict(x, div, rule, feature):
    '''
    输出单独层预测结果
    :param x: 预测样本
    :param div: 划分点
    :param rule: 划分规则
    :param feature: 进行操作的特征
    :return:
    '''
    if rule == 'LisOne':    #依据划分规则定义小于及大于划分点的标签    
        L = 1; H = -1
    else:                   
        L = -1; H = 1

    if x[feature] < div: 
        return L    #判断预测结果
    else:   
        return H

In [13]:
errorCnt = 0
for i in range(len(testData[:1000])):
    result = 0                         #预测结果值，初始为0
    for curTree in tree:               #依据算法8.1式8.6.预测式子是一个求和式，对于每一层的结果都要进行一次累加

        div = curTree['div']            #获取该层参数
        rule = curTree['rule']
        feature = curTree['feature']
        alpha = curTree['alpha']
        
        result += alpha * predict(testData[i], div, rule, feature)    #将当前层结果加入预测中
     
    if np.sign(result) != testLabel[i]: 
        errorCnt += 1   #预测结果取sign值，如果大于0 sign为1，反之为0
accuracy = 1 - errorCnt / 1000  #返回准确率
accuracy

0.0